In [35]:
# Ajoutez ceci avant les imports
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [36]:
# Importation des bibliothèques nécessaires
import os  # Pour gérer les fichiers et dossiers
from langchain_community.document_loaders import DirectoryLoader, TextLoader  # Pour charger les fichiers Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Pour découper les documents
from langchain_community.embeddings import HuggingFaceEmbeddings  # Pour générer les embeddings
from langchain_community.vectorstores import FAISS  # Pour stocker les embeddings
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings  # Pour utiliser le modèle mistral:7b
from langchain.chains import create_retrieval_chain  # Pour la chaîne de récupération
from langchain.chains.combine_documents import create_stuff_documents_chain  # Pour combiner les documents
from langchain_core.documents import Document  # Pour représenter les documents
from langchain.prompts import PromptTemplate  # Pour personnaliser le prompt
from sentence_transformers import SentenceTransformer, util  # Pour le re-ranking avec BAAI/bge-m3
import numpy as np  # Pour calculer les similarités
from langchain_core.retrievers import BaseRetriever


In [37]:
# ---------------------------------------
# SECTION 1 : Configuration
# ---------------------------------------

#paramètres importants pour le script
CONFIG = {
    "markdown_dir": "./markdown_branchements",  # Dossier des fichiers Markdown
    "faiss_dir": "./faiss_index",  # Dossier pour la base FAISS
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",  # Modèle pour l'indexation
    "reranker_model": "bge-m3:latest",  # Modèle pour le re-ranking
    "ollama_model": "mistral:7b",  # Modèle de langage
    "top_k_retrieve": 10,  # Documents à récupérer
    "top_k_rerank": 3,  # Documents après re-ranking
    "chunk_size": 512,  # Taille des morceaux
    "chunk_overlap": 50  # Chevauchement des morceaux
}


In [38]:
# ---------------------------------------
# SECTION 2 : Classe pour charger les fichiers
# ---------------------------------------
class RobustTextLoader(TextLoader):
    """Classe pour lire les fichiers Markdown avec gestion des encodages."""
    def __init__(self, file_path, encoding="utf-8", fallback_encodings=["iso-8859-1", "cp1252", "utf-16"]):
        super().__init__(file_path, encoding=encoding)
        self.fallback_encodings = fallback_encodings

    def lazy_load(self):
        for encoding in [self.encoding] + self.fallback_encodings:
            try:
                with open(self.file_path, encoding=encoding) as f:
                    text = f.read()
                yield Document(page_content=text, metadata={"source": self.file_path})
                return
            except UnicodeDecodeError:
                continue
        print(f"Échec du chargement de {self.file_path} : impossible de décoder.")

In [39]:
# ---------------------------------------
# SECTION 3 : Prompt personnalisé
# ---------------------------------------
# On définit un modèle de prompt pour que les réponses soient claires et structurées
PROMPT = PromptTemplate(
    template="""
Tu es un assistant expert du processus de branchement chez ENEO. Tes réponses doivent être :
- Précises et basées exclusivement sur les documents fournis
- En français courant et facile à comprendre
- Structurées avec des listes à puces quand c'est pertinent
- Précise le nom des documents d'où proviennent tes réponses

Contexte :
{context}

Question :
{input}

Réponds en t'appuyant sur le contexte fourni. Si tu ne sais pas, dis que tu n'as pas l'information.
""",
    input_variables=["context", "input"]
)

# Vérification du prompt pour diagnostiquer les erreurs
print("Prompt template utilisé :")
print(PROMPT.template)
print("Variables attendues :", PROMPT.input_variables)

Prompt template utilisé :

Tu es un assistant expert du processus de branchement chez ENEO. Tes réponses doivent être :
- Précises et basées exclusivement sur les documents fournis
- En français courant et facile à comprendre
- Structurées avec des listes à puces quand c'est pertinent
- Précise le nom des documents d'où proviennent tes réponses

Contexte :
{context}

Question :
{input}

Réponds en t'appuyant sur le contexte fourni. Si tu ne sais pas, dis que tu n'as pas l'information.

Variables attendues : ['context', 'input']


In [40]:
# ---------------------------------------
# SECTION 4 :  l'indexation
# ---------------------------------------
def setup_search_embeddings():
    """Configure le modèle d’embeddings pour la recherche."""
    print("Configuration du modèle d'embeddings...")
    return HuggingFaceEmbeddings(
        model_name=CONFIG["embedding_model"],
        model_kwargs={"device": "cpu"}
    )



In [41]:
# ---------------------------------------
# SECTION 5 :  chargement des documents
# ---------------------------------------

def load_documents():
    """Charge les fichiers Markdown."""
    print("Chargement des fichiers Markdown...")
    loader = DirectoryLoader(
        CONFIG["markdown_dir"],
        glob="**/*.md",
        loader_cls=RobustTextLoader,
        loader_kwargs={"encoding": "utf-8"},
        show_progress=True
    )
    documents = loader.load()
    print(f"{len(documents)} documents chargés :")
    for doc in documents:
        excerpt = doc.page_content[:100].replace("\n", " ")
        print(f" - {doc.metadata['source']} (extrait : {excerpt}...)")
    return documents



In [42]:

# ---------------------------------------
# SECTION 6 :  découpage des documents
# ---------------------------------------
def split_documents(documents):
    """Découpe les documents en morceaux."""
    print("Découpage des documents en morceaux...")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CONFIG["chunk_size"],
        chunk_overlap=CONFIG["chunk_overlap"]
    )
    chunks = splitter.split_documents(documents)
    print(f"{len(chunks)} morceaux créés.")
    return chunks




In [43]:
# ---------------------------------------
# SECTION 7 :  création de la base vectorielle
# ---------------------------------------
def create_vectorstore(chunks, embedding_model):
    """Crée et sauvegarde la base vectorielle FAISS."""
    print("Création de la base vectorielle...")
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(CONFIG["faiss_dir"])
    print(f"Base vectorielle sauvegardée dans {CONFIG['faiss_dir']}.")
    return vectorstore




In [44]:

# ---------------------------------------
# SECTION 8 :  chargement de la base vectorielle
# ---------------------------------------
def load_vectorstore(embedding_model):
    """Charge la base vectorielle FAISS existante."""
    print(f"Chargement de la base vectorielle depuis {CONFIG['faiss_dir']}...")
    vectorstore = FAISS.load_local(CONFIG["faiss_dir"], embedding_model, allow_dangerous_deserialization=True)
    print("Base vectorielle chargée.")
    return vectorstore




In [45]:
# ---------------------------------------
# SECTION 9 :  configuration du re-ranking
# ---------------------------------------

def setup_reranker():
    """Configure le modèle de re-ranking avec Ollama."""
    print("Configuration du re-ranker...")
    return OllamaEmbeddings(model=CONFIG["reranker_model"])



In [46]:

# ---------------------------------------
# SECTION 10 :  création du retriever (récupère les doc de ma base et les reclasse avec reranker_model)
# ---------------------------------------

def create_custom_retriever(vectorstore, reranker_model):
    """Crée un retriever avec re-ranking compatible """
    class CustomRetriever(BaseRetriever):
        vectorstore: FAISS
        reranker_model: OllamaEmbeddings
        top_k_retrieve: int
        top_k_rerank: int

        def _get_relevant_documents(self, query: str) -> list[Document]:
            # Récupérer les documents initiaux
            initial_docs = self.vectorstore.similarity_search(query, k=self.top_k_retrieve)
            # Générer l’embedding de la question
            query_embedding = np.array(self.reranker_model.embed_query(query))
            # Générer les embeddings des documents
            doc_embeddings = np.array([self.reranker_model.embed_query(doc.page_content) for doc in initial_docs])
            # Calculer les similarités cosinus
            similarities = np.dot(doc_embeddings, query_embedding) / (
                np.linalg.norm(doc_embeddings, axis=1) * np.linalg.norm(query_embedding)
            )
            # Trier par similarité
            scored_docs = list(zip(initial_docs, similarities))
            scored_docs.sort(key=lambda x: x[1], reverse=True)
            # Retourner les meilleurs documents
            return [doc for doc, _ in scored_docs[:self.top_k_rerank]]

    return CustomRetriever(
        vectorstore=vectorstore,
        reranker_model=reranker_model,
        top_k_retrieve=CONFIG["top_k_retrieve"],
        top_k_rerank=CONFIG["top_k_rerank"]
    )



In [47]:
# ---------------------------------------
# SECTION 11 :  configuration du LLM
# ---------------------------------------
def setup_llm():
    """Configure le modèle de langage."""
    print(f"Configuration du LLM {CONFIG['ollama_model']}...")
    llm = Ollama(model=CONFIG["ollama_model"])
    print("LLM configuré.")
    return llm




In [48]:

# ---------------------------------------
# SECTION 12 :  création de la chaîne QA
# ---------------------------------------

def create_chain(llm, retriever):
    """Crée la chaîne de question-réponse."""
    print("Création de la chaîne de question-réponse...")
    combine_docs_chain = create_stuff_documents_chain(llm, PROMPT)
    chain = create_retrieval_chain(retriever, combine_docs_chain)
    print("Chaîne créée avec succès.")
    return chain

# test de la chaine

def test_chain(chain, question):
    """Teste la chaîne avec une question."""
    print(f"\nQuestion : {question}")
    try:
        response = chain.invoke({"input": question})
        print(f"Réponse : {response['answer']}")
    except Exception as e:
        print(f"Erreur lors de l'invocation de la chaîne : {str(e)}")




In [49]:
# ---------------------------------------
# SECTION 13 :  interactoin avec le LLM
# ---------------------------------------


def interactive_mode(chain):
    """Lance le mode interactif pour poser des questions."""
    print("\nMode interactif activé. Tapez 'quitter' pour arrêter.")
    while True:
        try:
            question = input("\nPosez une question : ")
            if question.lower() == "quitter":
                break
            print(f"Question : {question}")
            response = chain.invoke({"input": question})
            print(f"Réponse : {response['answer']}")
        except Exception as e:
            print(f"Erreur dans le mode interactif : {str(e)}")
            continue

In [50]:
# ---------------------------------------
# SECTION 14 :  main
# ---------------------------------------
def main():
    """Orchestre le programme avec toutes les  fonctions précedemment définies."""
    try:
        # Configurer les composants
        embedding_model = setup_search_embeddings()
        
        # Charger ou créer la base vectorielle
        vectorstore = (load_vectorstore(embedding_model) if os.path.exists(CONFIG["faiss_dir"])
                       else create_vectorstore(split_documents(load_documents()), embedding_model))
        
        # Configurer le re-ranker et le retriever
        reranker_model = setup_reranker()
        retriever = create_custom_retriever(vectorstore, reranker_model)
        
        # Configurer le LLM et la chaîne
        llm = setup_llm()
        chain = create_chain(llm, retriever)
        
        # Tester et passer en mode interactif
        test_chain(chain, "decris moi la marche à suivre pour effectuer un nouveau branchement ?")
        interactive_mode(chain)
        
    except Exception as e:
        print(f"Une erreur est survenue : {str(e)}")
    finally:
        print("\nProgramme terminé.")

In [51]:
# ---------------------------------------
# SECTION 15 :  lancement du programme
# ---------------------------------------
if __name__=="__main__":
    main()


Configuration du modèle d'embeddings...
Chargement de la base vectorielle depuis ./faiss_index...
Base vectorielle chargée.
Configuration du re-ranker...
Configuration du LLM deepseek-r1:14b...
LLM configuré.
Création de la chaîne de question-réponse...
Chaîne créée avec succès.

Question : decris moi la marche à suivre pour effectuer un nouveau branchement ?
Réponse : <think>
Okay, so I'm trying to figure out the process for a new branch connection based on the information provided. Let me start by looking at the context given.

The context mentions two main points:

1. Every month, the company must send detailed statements of connections made and the amounts spent by them for these works no later than the 15th of the following month.
2. They need to recover the amounts from households connected as per the revolving fund agreement signed by the beneficiary.

Additionally, there's a commercial component related to new prepaid connections.

So, based on this, I can outline some steps, b